In [ ]:
! ! pip install -q -U transformers==4.41.2 peft==0.11.1 trl==0.8.6 accelerate==0.30.1 bitsandbytes==0.43.1 datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 75.1 MB/s eta 0:00:00
ERROR: pip's 

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [2]:
from datasets import load_dataset

In [10]:
# Loading dataset

dataset=load_dataset("ruslanmv/ai-medical-chatbot",split="all")

In [15]:
dataset=dataset.shuffle(seed=42).select(range(500))


In [54]:
def template_format(dataset):
  new_formated=[]
  for data in dataset:
    # temp={"prompt":data["Patient"] ,"completion":data["Doctor"]}

    temp={"messages":
      [
         {"role": "system", "content": "Patient-Doctor Chat."},
         {"role": "user", "content": data["Patient"]},
         {"role": "assistant", "content": data["Doctor"]}
     ]
    }

    new_formated.append(temp)
  return new_formated

In [55]:
new_data=template_format(dataset)

In [60]:
new_data # Converted formated fot gpt-3.5 finetuining

In [57]:
# Convert our data to jsonl file
import json
with open("docter_patient_data.jsonl","w") as file:
  for items in new_data:
    file.write(json.dumps(items)+"\n")


In [43]:
from openai import OpenAI
client=OpenAI(api_key="your key")

In [58]:
client.files.create(
    file=open("docter_patient_data.jsonl","rb"),
    purpose="fine-tune"
)
#every thing is created . it may take some time to create if the file size is large

FileObject(id='file-BBB0gEJXJ5yACnEuBJQ7dDPx', bytes=568755, created_at=1718387026, filename='docter_patient_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [59]:
# Create fine tuned model. once the files created we should give the file id
client.fine_tuning.jobs.create(
    training_file="file-BBB0gEJXJ5yACnEuBJQ7dDPx",
    model="gpt-3.5-turbo"
)


FineTuningJob(id='ftjob-TrSbMg5SFfuqUwHOdFdwDOzF', created_at=1718387046, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-IHDI1nXLTH3K8ORN0cnQIoiP', result_files=[], seed=235956206, status='validating_files', trained_tokens=None, training_file='file-BBB0gEJXJ5yACnEuBJQ7dDPx', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
# It will take 1hr or more we can see the status of finetune in their website

In [ ]:
# we will get the model id after finetune id which we can use to access the model
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="finetune model id",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)
print(completion.choices[0].message)